<a href="https://colab.research.google.com/github/shreejitachauhan/Data_engineering/blob/transform/transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install snowflake-connector-python
!pip install "snowflake-connector-python[pandas]"

import snowflake.connector
import getpass

password = getpass.getpass("Enter Snowflake password: ")
conn = snowflake.connector.connect(
    user="SHREEJITACHAUHAN",
    password=password,
    account="FHTJQHV-WW96608",
    role="ACCOUNTADMIN",
    warehouse="COMPUTE_WH",
    database="DATASET",
    schema="SALES"
)
cursor = conn.cursor()

cursor.execute("CREATE OR REPLACE SCHEMA sales_temp;")

cursor.execute("""
CREATE OR REPLACE TABLE sales_temp.Dim_Date (
    Date_ID STRING PRIMARY KEY,
    DAY INT, MONTH INT, YEAR INT,
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    UPDATED_AT TIMESTAMP
);
""")
cursor.execute("""
CREATE OR REPLACE TABLE sales_temp.Dim_Branch (
    Branch_ID STRING PRIMARY KEY,
    BRANCH_NAME STRING,
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    UPDATED_AT TIMESTAMP
);
""")
cursor.execute("""
CREATE OR REPLACE TABLE sales_temp.Dim_Dealer (
    Dealer_ID STRING PRIMARY KEY,
    DEALER_NAME STRING,
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    UPDATED_AT TIMESTAMP
);
""")
cursor.execute("""
CREATE OR REPLACE TABLE sales_temp.Dim_Product (
    Model_ID STRING PRIMARY KEY,
    PRODUCT_NAME STRING,
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    UPDATED_AT TIMESTAMP
);
""")
cursor.execute("""
CREATE OR REPLACE TABLE sales_temp.Fact_Sales (
    Date_ID STRING, Branch_ID STRING, Dealer_ID STRING, Model_ID STRING,
    Revenue INT, Units_Sold INT,
    Created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")

cursor.execute("""
INSERT INTO sales_temp.Dim_Date (Date_ID, DAY, MONTH, YEAR)
SELECT DISTINCT DATE_ID, DAY, MONTH, YEAR
FROM "sales_staging"."staging_sales"
WHERE DATE_ID IS NOT NULL
  AND DATE_ID NOT IN (SELECT Date_ID FROM sales_temp.Dim_Date);
""")

cursor.execute("""
INSERT INTO sales_temp.Dim_Branch (Branch_ID, BRANCH_NAME)
SELECT DISTINCT BRANCH_ID, BRANCHNAME
FROM "sales_staging"."staging_sales"
WHERE BRANCH_ID IS NOT NULL
  AND BRANCH_ID NOT IN (SELECT Branch_ID FROM sales_temp.Dim_Branch);
""")

cursor.execute("""
INSERT INTO sales_temp.Dim_Dealer (Dealer_ID, DEALER_NAME)
SELECT DISTINCT DEALER_ID, DEALERNAME
FROM "sales_staging"."staging_sales"
WHERE DEALER_ID IS NOT NULL
  AND DEALER_ID NOT IN (SELECT Dealer_ID FROM sales_temp.Dim_Dealer);
""")

cursor.execute("""
INSERT INTO sales_temp.Dim_Product (Model_ID, PRODUCT_NAME)
SELECT DISTINCT MODEL_ID, PRODUCT_NAME
FROM "sales_staging"."staging_sales"
WHERE MODEL_ID IS NOT NULL
  AND MODEL_ID NOT IN (SELECT Model_ID FROM sales_temp.Dim_Product);
""")

cursor.execute("""
INSERT INTO sales_temp.Fact_Sales (
    Date_ID, Branch_ID, Dealer_ID, Model_ID, Revenue, Units_Sold
)
SELECT s.DATE_ID, s.BRANCH_ID, s.DEALER_ID, s.MODEL_ID, s.REVENUE, s.UNITS_SOLD
FROM "sales_staging"."staging_sales" s
JOIN sales_temp.Dim_Date d ON s.DATE_ID = d.Date_ID
JOIN sales_temp.Dim_Branch b ON s.BRANCH_ID = b.Branch_ID
JOIN sales_temp.Dim_Dealer dl ON s.DEALER_ID = dl.Dealer_ID
JOIN sales_temp.Dim_Product p ON s.MODEL_ID = p.Model_ID;
""")

Enter Snowflake password: ··········
